In [3]:
import os
import numpy as np
from PIL import Image

In [19]:
data_dir = 'datasets/bdd/seg/labels/val'

In [21]:
for jpg in os.listdir(data_dir):
    image = Image.open(os.path.join(data_dir, jpg))
    image = np.array(image)
    print(jpg)
    print(image.shape)
    break

aaddc191-6f4e53c1_train_id.png
(720, 1280)
